## CNN for MNIST

In [45]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


### Training

In [14]:
tf.reset_default_graph()

#parameters
learning_rate = 0.01
training_epochs = 50
batch_size = 100
num = mnist.train.num_examples//batch_size

# input placeholder
x = tf.placeholder(tf.float32, [None, 784])
x_img = tf.reshape(x, [-1,28,28,1])
y = tf.placeholder(tf.float32, [None, 10])

# feature extraction
w1 = tf.Variable(tf.random_normal([3,3,1,32], stddev = 0.01))
                                      # stddev -> 표준편차 0.01의 정규분포
l1 = tf.nn.conv2d(x_img, w1, strides = [1,1,1,1], padding="SAME")
l1 = tf.nn.relu(l1)
l1 = tf.nn.max_pool(l1, ksize = [1,2,2,1], strides = [1,2,2,1],
                   padding = "SAME") # [?, 14,14,32]
"[?,28,28,1] -> [?, 28, 28, 32] -> [?,14,14,32]"
                            # feature을 덜 세세하게 함으로써 overfitting 을 방지

w2 = tf.Variable(tf.random_normal([3,3,32,64], stddev = 0.01)) # [?, 14,14, 32]
l2 = tf.nn.conv2d(l1, w2, strides = [1,1,1,1], padding = "SAME") 
l2 = tf.nn.relu(l2)
l2 = tf.nn.max_pool(l2, ksize = [1,2,2,1], strides = [1,2,2,1], # [?,7,7,32]
                   padding="SAME")
l2 = tf.reshape(l2, [-1, 7*7*64]) # FC 를 위해 쫙 펼침
"[?,14,14,32] -> [?,14,14,64] -> [?,14,14,64] -> [?,14*14*64]"

w3 = tf.get_variable("w3", shape=[7*7*64,10],
                    initializer = tf.contrib.layers.xavier_initializer())
b = tf.get_variable("b3", shape = [10],
                   initializer = tf.contrib.layers.xavier_initializer())

# classification
hypothesis = tf.matmul(l2,w3)+b

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits
                      (logits = hypothesis, labels = y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

writer = tf.summary.FileWriter("./MNIST_graph")

sess = tf.Session()
sess.run(tf.global_variables_initializer())
writer.add_graph(sess.graph)

for epoch in range(training_epochs):
    avg_cost = 0
    for step in range(num):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {x:batch_xs, y:batch_ys}
        c, _= sess.run([cost,optimizer], feed_dict = feed_dict)
        avg_cost += c/num
    print("Epoch:", "%04d"%(epoch+1), "cost =", "{:011.9f}".format(avg_cost))







Epoch: 0001 cost = 0.139091453
Epoch: 0002 cost = 0.054334230
Epoch: 0003 cost = 0.039152828
Epoch: 0004 cost = 0.033912304
Epoch: 0005 cost = 0.027577225
Epoch: 0006 cost = 0.027127721
Epoch: 0007 cost = 0.023791519
Epoch: 0008 cost = 0.023780140
Epoch: 0009 cost = 0.024759805
Epoch: 0010 cost = 0.020484630
Epoch: 0011 cost = 0.017005853
Epoch: 0012 cost = 0.020540934
Epoch: 0013 cost = 0.023551873
Epoch: 0014 cost = 0.014639197
Epoch: 0015 cost = 0.029460054
Epoch: 0016 cost = 0.016297161
Epoch: 0017 cost = 0.015860078
Epoch: 0018 cost = 0.017379001
Epoch: 0019 cost = 0.023954951
Epoch: 0020 cost = 0.018790967
Epoch: 0021 cost = 0.015032721
Epoch: 0022 cost = 0.019075310
Epoch: 0023 cost = 0.016969801
Epoch: 0024 cost = 0.018007222
Epoch: 0025 cost = 0.015717998
Epoch: 0026 cost = 0.017858515
Epoch: 0027 cost = 0.018112548
Epoch: 0028 cost = 0.017693828
Epoch: 0029 cost = 0.017721742
Epoch: 0030 cost = 0.033677880
Epoch: 0031 cost = 0.024831109
Epoch: 0032 cost = 0.018849633
Epoch: 0

### Evaluation

In [19]:
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, dtype = tf.float32))
a = sess.run(accuracy, feed_dict={x:mnist.test.images,
                                                y:mnist.test.labels})
print("accuracy : {:10.5%}".format(a))

accuracy :  98.75000%


## Deep CNN

In [53]:
tf.reset_default_graph()

# l1 image shape=(?,28,28,1)
#parameters
training_epochs = 10
learning_rate = 0.01
batch_size = 100
num = mnist.train.num_examples//batch_size

# Placeholers
x = tf.placeholder(tf.float32, [None,784])
x_img = tf.reshape(x, [-1, 28,28,1])
y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)

w1 = tf.Variable(tf.random_normal([3,3,1, 32], stddev = 0.01))
b1 = tf.Variable(tf.random_normal([32], stddev = 0.01)) # 32개의 filter
l1 = tf.nn.conv2d(x_img, w1, strides = [1,1,1,1], padding = "SAME")+b1
l1 = tf.nn.relu(l1)
l1 = tf.nn.max_pool(l1, ksize = [1,2,2,1], strides = [1,2,2,1],
                   padding = "SAME") # 원래는 이게 input
l1 = tf.nn.dropout(l1, keep_prob = keep_prob)

# l2 image shape=(?,14,14,32)
w2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
b2 = tf.Variable(tf.random_normal([64]))
l2 = tf.nn.conv2d(l1, w2, strides = [1,1,1,1], padding = "SAME")+b2
l2 = tf.nn.relu(l2)
l2 = tf.nn.max_pool(l2, ksize = [1,2,2,1], strides=[1,2,2,1],
                   padding = "SAME")
l2 = tf.nn.dropout(l2, keep_prob = keep_prob)

# l3 image shape=(?,7,7,64)
w3 = tf.Variable(tf.random_normal([3,3,64,128], stddev = 0.01))
b3 = tf.Variable(tf.random_normal([128], stddev = 0.01))
l3 = tf.nn.conv2d(l2, w3, strides = [1,1,1,1], padding="SAME")+b3
l3 = tf.nn.relu(l3)
l3 = tf.nn.max_pool(l3, ksize = [1,2,2,1], strides = [1,2,2,1],
                   padding="SAME")
l3 = tf.nn.dropout(l3, keep_prob= keep_prob)
l3= tf.reshape(l3, [-1,4*4*128])

# classification

W1 = tf.get_variable("W1", shape = [4*4*128,625],
                        initializer = tf.contrib.layers.xavier_initializer())
B1 = tf.get_variable("B11", shape = [625],
                        initializer = tf.contrib.layers.xavier_initializer())
L1 = tf.matmul(l3,W1)+B1
L1 = tf.nn.relu(L1)
L1 = tf.nn.dropout(L1, keep_prob = keep_prob)

W2 = tf.get_variable("W2", shape = [625,10],
                        initializer = tf.contrib.layers.xavier_initializer())
B2 = tf.get_variable("B2", shape = [10],
                        initializer = tf.contrib.layers.xavier_initializer())

hypothesis = tf.matmul(L1, W2) + B2
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis,
                                labels = y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(training_epochs):
    avg_cost = 0
    for step in range(num):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        c, _= sess.run([cost,optimizer], feed_dict = {x:batch_xs, y:batch_ys,
                      keep_prob:0.7})
        avg_cost += c/num
    print("Epoch:", "%04d"%(epoch+1), "cost =", "{:.9f}".format(avg_cost))

Epoch: 0001 cost = 0.744329727
Epoch: 0002 cost = 0.348777859
Epoch: 0003 cost = 0.328508723
Epoch: 0004 cost = 0.293287922
Epoch: 0005 cost = 0.281662331
Epoch: 0006 cost = 0.268073255
Epoch: 0007 cost = 0.270948216
Epoch: 0008 cost = 0.259456737
Epoch: 0009 cost = 0.274750360
Epoch: 0010 cost = 0.274909263


In [ ]:
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, dtype = tf.float32))
a = sess.run(accuracy, feed_dict={x:mnist.test.images,
                                                y:mnist.test.labels,
                                 keep_prob:1})
print("accuracy : {:10.5%}".format(a))